weaves

blkswn data exercise

In [ ]:
import os
import datetime

import pandas as pd
from pandas.io.json import json_normalize

import numpy as np
import functools
import scipy.special as scis
from functools import partial
from itertools import permutations, combinations, chain
import string

import json
import ast
import sys

import re

pd.__version__

%load_ext autoreload
%autoreload 2

In [ ]:
import glob
[f for f in glob.glob("data" + "/*", recursive=True)] # list files for reference

In [ ]:
# Load with pandas as CSV, check the data frame dimensions, check the column names are the same.
df1 = pd.read_csv('data/csv_file.csv', encoding = 'utf8')
df2 = pd.read_csv('data/txt_file.txt', encoding = 'utf8')
print((df1.shape, df2.shape))
set(df1.columns.values).difference(df2.columns.values)

In [ ]:
df = df1.append(df2, ignore_index=True) # and row catenate, re-index the appended dataframe
df.shape
df.columns.values

In [ ]:
df

In [ ]:
# Load JSON file - real JSON - has true and false not 
flnm = 'data/json_file.json'

In [ ]:
# Nested JSON
df3 = pd.read_json(flnm, encoding="utf-8")
print(df3.shape)
df3

In [ ]:
# Try a JSON load and normalize, preserves embedded UTF-8
with open(flnm, encoding="utf-8") as f:
    d0 = json.load(f)
print(len(d0), type(d0))
d0

In [ ]:
df3 = json_normalize(d0)
print(df3.shape)
df3.head(3)
set(df.columns.values).difference(set(df3.columns.values))

In [ ]:
# So same column names! Makes it easier.
df = df.append(df3, ignore_index=True)
df.shape

In [ ]:
df.info()

In [ ]:
df.to_pickle("./bak/blkswn.pkl")
df.to_json("./bak/blkswn.json", orient='records')

df1 = None; df2 = None; df3 = None

In [ ]:
# Use this as a backup
xdf = pd.read_pickle("./bak/blkswn.pkl")
print(xdf.shape)
xdf.head(3)

In [ ]:
df.info()
df.head(3)

One bad record in the text file: search 'TELL ME'. And a bad platform.
Force some strings to be categories. Sentiment appears to be factor.

In [ ]:
df = df[df['content.body'].notnull()]

In [ ]:
v0 = [ x for x in set(df['properties.platform']) if isinstance(x, str)]
v0

In [ ]:
i0 = df.index[df['properties.platform'].isnull()].tolist()
df.loc[i0]

In [ ]:
df.loc[i0,'properties.platform'] = v0[0]
df.loc[i0]

In [ ]:
df['author.properties.friends'] = df['author.properties.friends'].astype(int)

In [ ]:
df['author.properties.verified'] = df['author.properties.verified'].astype(bool)

In [ ]:
df['author.properties.status_count'] = df['author.properties.status_count'].astype(int)

In [ ]:
df['properties.platform'] = df['properties.platform'].astype('category')

In [ ]:
df['location.country'] = df['location.country'].astype('category')

In [ ]:
set(df['properties.sentiment'].tolist())

In [ ]:
df['properties.sentiment'] = df['properties.sentiment'].astype('int')

In [ ]:
zdf = pd.DataFrame({'value': np.random.randint(0, 100, 20)})
labels = [ "{0} - {1}".format(i, i + 9) for i in range(0, 100, 10) ]
zdf['group'] = pd.cut(zdf.value, range(0, 105, 10), right=False, labels=labels)

In [ ]:
list(range(0, 105, 10))

In [ ]:
v1 = list(set(df['properties.sentiment'].tolist()))
v1.sort()
v1

In [ ]:
zdf = df

labels=('ugly', 'bad', 'good')
zdf['s'] = pd.cut(zdf['properties.sentiment'], range(-1, 3), right=False, labels=labels)

In [ ]:
zdf[['properties.sentiment', 's']].info()
zdf[['properties.sentiment', 's']].head(3)

In [ ]:
s1 = pd.Categorical(zdf['s'], categories=["ugly", "bad", "good"], ordered=True)
s1

In [ ]:
zdf['s1'] = s1

In [ ]:
zdf[['properties.sentiment', 's', 's1']].head(8)

In [ ]:
zdf = zdf.drop(['s'], axis=1)
zdf.head(8)
df = zdf
zdf = None
df.to_pickle("./bak/blkswn1.pkl")
df.to_json("./bak/blkswn1.json", orient='records')

In [ ]:
df.head(3)

In [ ]:
# A utility transformer - spot tags
def trns0(x, fld=None):
    return x[fld].count('@')

trns1 = partial(trns0, fld='content.body')
    
df['at.n'] = df.apply(trns1, axis=1)

In [ ]:
# A utility transformer - spot tags
def trns0(x, fld=None):
    return x[fld].count('#')

trns1 = partial(trns0, fld='content.body')
    
df['hash.n'] = df.apply(trns1, axis=1)

In [ ]:
# A utility transformer - word count
def trns0(x, fld=None):
    return len(x[fld].split())

trns1 = partial(trns0, fld='content.body')
    
df['words.n'] = df.apply(trns1, axis=1)

In [ ]:
# A utility transformer - emojis

emoticons = set(range(int('1f600',16), int('1f650', 16)))

def split_count(text):
    n0 = 0
    for ch in text:
        if ord(ch) in emoticons:
            n0 += 1
    return n0

def trns0(x, fld=None):
    return split_count(x[fld])

trns1 = partial(trns0, fld='content.body')
    
df['emoji.n'] = df.apply(trns1, axis=1)

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def isEnglish(s):
    s = emoji_pattern.sub(r'', s) # no emoji
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
        
def trns0(x, fld=None):
    return isEnglish(x[fld])

trns1 = partial(trns0, fld='content.body')
    
df['english.q'] = df.apply(trns1, axis=1)

In [ ]:
df[df['english.q'] == False]

In [ ]:
df.columns.values

In [ ]:
# Make training and test sets
y=df['s1'] # outcome
X=df.drop(['s1', 'properties.sentiment', 'content.body', 'properties.platform'], axis=1)
X=pd.get_dummies(X, columns =  ['location.country'])

In [ ]:
X.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
X1 = scaler.transform(X_train) # 1 is known
X0 = scaler.transform(X_test)  # is unknown

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X1, y_train)

In [ ]:
predictions = mlp.predict(X0)

In [ ]:
y_test.value_counts()

In [ ]:
pd.Series(predictions).value_counts()

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,predictions))

In [ ]:
print(classification_report(y_test,predictions))